## 2RPFS Problem (TWCT objective) - Tables and Graphs

Before running this, notebook, please run notebook 0.1.

In [ ]:
import pandas as pd
import numpy as np
import os, fnmatch
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
import glob
import seaborn as sns
import gzip
import matplotlib.style as style
from matplotlib.path import Path
from matplotlib.patches import BoxStyle

%matplotlib inline

In [ ]:
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

In [ ]:
linestyle_tuple = [
     ('dotted',                (0, (1, 1))),
     ('dashed',                (0, (5, 5))),
     ('densely dashed',        (0, (5, 1))),
     ('dashdotdotted',         (0, (3, 5, 1, 5, 1, 5))),
     ('densely dashdotdotted', (0, (3, 1, 1, 1, 1, 1))),

     ('dashdotted',            (0, (3, 5, 1, 5))),
     ('densely dashdotted',    (0, (3, 1, 1, 1))),
     
     ('loosely dashed',        (0, (5, 10))),
     ('loosely dashdotted',    (0, (3, 10, 1, 10))),
     

     ('loosely dashdotdotted', (0, (3, 10, 1, 10, 1, 10))),
     ('densely dotted',        (0, (1, 1))),
     ('loosely dotted',        (0, (1, 10)))]

In [ ]:
# https://stackoverflow.com/questions/51483901/is-there-a-way-to-extend-the-solid-color-background-to-the-full-width-of-the-pag
class ExtendedTextBox(BoxStyle._Base):
    """
    An Extended Text Box that expands to the axes limits 
                        if set in the middle of the axes
    """

    def __init__(self, pad=0.3, width=500.):
        """
        width: 
            width of the textbox. 
            Use `ax.get_window_extent().width` 
                   to get the width of the axes.
        pad: 
            amount of padding (in vertical direction only)
        """
        self.width=width
        self.pad = pad
        super(ExtendedTextBox, self).__init__()

    def transmute(self, x0, y0, width, height, mutation_size):
        """
        x0 and y0 are the lower left corner of original text box
        They are set automatically by matplotlib
        """
        # padding
        pad = mutation_size * self.pad

        # we add the padding only to the box height
        height = height + 2.*pad
        # boundary of the padded box
        y0 = y0 - pad
        y1 = y0 + height
        _x0 = x0
        x0 = _x0 +width /2. - self.width/2.
        x1 = _x0 +width /2. + self.width/2.

        cp = [(x0, y0),
              (x1, y0), (x1, y1), (x0, y1),
              (x0, y0)]

        com = [Path.MOVETO,
               Path.LINETO, Path.LINETO, Path.LINETO,
               Path.CLOSEPOLY]

        path = Path(cp, com)

        return path

### List files in the result folder 

In [ ]:
resultfolder = os.path.join(os.getcwd(), 'results', 'consolidated')
rpfs_file = os.path.join(resultfolder, 'RPFS_TWCT_all_results.pkl.gz')

### Create the output folder 

In [ ]:
outputfolder = os.path.join(os.getcwd(), 'results', 'consolidated')
outputfolder_graph = os.path.join(os.getcwd(), 'results', 'consolidated', 'graphs')
outputfolder_table = os.path.join(os.getcwd(), 'results', 'consolidated', 'tables')
if not os.path.exists(outputfolder_graph):
    os.makedirs(outputfolder_graph)
if not os.path.exists(outputfolder_table):
    os.makedirs(outputfolder_table)
#print('Saving files on folder: ' + outputfolder)

### Process consolidated CSV result files

In [ ]:
df_rpfs = pd.read_pickle(rpfs_file)  # Robust PFSP Budget solutions only
df_rpfs.drop(columns=['executionId'], inplace=True)
df_rpfs = df_rpfs.reset_index()

**Robust dataframe: calculating new fields.**

In [ ]:
df_rpfs['optimal'] = df_rpfs['is_optimal'] & df_rpfs['validated'] & (df_rpfs['gap'] <= 1e-8)
df_rpfs['time_limit'] = 7200.0
df_rpfs['time_limit_2'] = 7200.0 * 2
df_rpfs['mp_total_time'] = (df_rpfs['n'] < 15).astype(int) * np.minimum(df_rpfs['mp_total_time'], df_rpfs['time_limit']) + (df_rpfs['n'] >= 15).astype(int) * np.minimum(df_rpfs['mp_total_time'], df_rpfs['time_limit_2'])
df_rpfs['time'] = df_rpfs['mp_total_time'] + df_rpfs['sp_total_time']
df_rpfs['gap'] = df_rpfs['gap'] * 100.0
df_rpfs['RobCost_worstcase'] = df_rpfs['wct_validation']
df_rpfs = df_rpfs.rename(columns={"budget_Gamma": "RobCost_Gamma"})

In [ ]:
df_rpfs.tail(4)

In [ ]:
df_rpfs.info()

### Checking the Robust PFSP Budget solutions dataframe

In [ ]:
df_rpfs.head(2)

# Tables

Replace model names with the name used in table presentation:

In [ ]:
df_rpfs = df_rpfs[(df_rpfs['model'] != 'hybrid')]
df_rpfs['model'].replace({'hybrid-liao-you': 'Liao-You-Hybrid', 'hybrid-wilson': 'Wilson-Hybrid', 'liao-you': 'Liao-You', 'manne': 'Manne',
                    'tba': 'TBA', 'ts2': 'TS2', 'ts3': 'TS3', 'wagner-wst2': 'WST2', 'wilson': 'Wilson'}, inplace=True)

Obtain list of C&CG models, instance types

In [ ]:
model_list = df_rpfs['model'].unique().tolist()
instance_type_list = df_rpfs['instance_type'].unique().tolist()
print(model_list)
print(instance_type_list)

Add a new column containing the instance size as string

In [ ]:
df_temp = df_rpfs
(df_temp['n'].astype(str) + 'x' + df_temp['m'].astype(str)).unique()

In [ ]:
df_rpfs.columns

In [ ]:
df_temp = df_rpfs
df_temp['instance_size'] = df_temp['n'].astype(str) + 'x' + df_temp['m'].astype(str)
df_rpfs = df_temp.set_index(['model', 'n', 'm', 'alpha', 'seq', 'RobCost_Gamma', 'instance_type'])
df_rpfs

Treating errors in the `gap` column

In [ ]:
df_rpfs['gap'].describe()

In [ ]:
df_check = df_rpfs.reset_index()[['model', 'n', 'm', 'alpha', 'seq', 'RobCost_Gamma', 'instance_name', 'gap', 'wct', 'RobCost_worstcase', 'lb']]
df_check[(df_check['gap'] < -1e-5)].to_csv(os.path.join(os.getcwd(), 'results', 'negative_gap_list.csv'))

In [ ]:
df_rpfs['gap'] = df_rpfs['gap'].apply(lambda x: np.maximum(x, 0.0))

In [ ]:
df_rpfs['gap'].describe()

## Table 2. Performance given all instances 

Model-wise Robust PFSP C&CG performance comparison, given all instances.

* % Best Performance is the percentage of instances solved to optimality where the model achieved shorter execution time, when compared to the other models; 

* % Solved contains the percentage of instances solved within the time limit; 

* % Solved < n x m > represents the percentage of solved instances of size n x m; 

* Avg. % Gap is the average percentage gap of solutions from instances not solved to optimality; 

* Median time is the median execution time, in seconds; 

* Median iterations is the median of the number of iterations performed.

In [ ]:
df_model = df_rpfs.reset_index()
df_model = df_model[df_model['model'] == 'hybrid']
df_model = df_model[df_model['optimal'] == True]    
df_model = df_model.set_index(['n', 'm', 'alpha', 'seq', 'RobCost_Gamma', 'instance_type'])

df_others = df_rpfs.reset_index()
df_others = df_others[df_others['model'] != 'hybrid']
df_others = df_others[df_others['optimal'] == True] 
group_columns = ['n', 'm', 'alpha', 'seq', 'RobCost_Gamma', 'instance_type']
df_best_performance = df_others[group_columns + ['time']].groupby(by=group_columns).min()['time']
df_best_performance = df_best_performance.to_frame()
df_best_performance

In [ ]:
def calculate_perc_best_performance(df, model):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    df_model = df_model[df_model['optimal'] == True]    
    df_model = df_model.set_index(['n', 'm', 'alpha', 'seq', 'RobCost_Gamma', 'instance_type'])
    if len(df_model.index) == 0:
        return np.nan
    
    df_others = df.reset_index()
    df_others = df_others[df_others['model'] != model]
    df_others = df_others[df_others['optimal'] == True] 
    group_columns = ['n', 'm', 'alpha', 'seq', 'RobCost_Gamma', 'instance_type']
    df_best_performance = df_others[group_columns + ['time']].groupby(by=group_columns).min()['time']
    df_best_performance = df_best_performance.to_frame()
    if len(df_best_performance.index) == 0:
        return np.nan
    
    df_compare = df_best_performance.join(df_model, how='inner', 
                                                     on=group_columns,
                                                     lsuffix='_best')
    df_compare['time_wins'] = (df_compare['time'] < df_compare['time_best']).astype(int)
    return np.round(100.0 * df_compare['time_wins'].sum() / len(df_compare.index), 2)

In [ ]:
def calculate_perc_solved(df, model, instance_type = None, instance_size = None):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    df_ = df_model
    if instance_type is not None:
        df_ = df_[df_['instance_type'] == instance_type]
    if instance_size is not None:
        df_ = df_[df_['instance_size'] == instance_size]
    if len(df_.index) > 0:
        return np.round(100.0 * len(df_[(df_['optimal'] == True)].index) / len(df_.index), 2)
    else:
        return np.nan

In [ ]:
# Avg. % Gap is the average percentage gap of solutions from instances not solved to optimality
def calculate_avg_perc_gap(df, model):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    df_model = df_model[df_model['optimal'] == False]
    return np.round(df_model['gap'].mean(), 2)

In [ ]:
def calculate_median_time(df, model, time_col_name):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    return np.round(df_model[time_col_name].median(), 2)

In [ ]:
def calculate_avg_time(df, model, time_col_name):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    return np.round(df_model[time_col_name].mean(), 2)

In [ ]:
def calculate_std_time(df, model, time_col_name):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    return np.round(df_model[time_col_name].std(), 2)

In [ ]:
def calculate_median_iterations(df, model):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    return np.round(df_model['iterations'].median(), 2)

In [ ]:
def calculate_avg_iterations(df, model):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    return np.round(df_model['iterations'].mean(), 2)

In [ ]:
def calculate_std_iterations(df, model):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    return np.round(df_model['iterations'].std(), 2)

In [ ]:
model_stats = dict()
for model in model_list:
    model_stats[model] = dict()
    model_stats[model]['% Best Performance'] = calculate_perc_best_performance(df_rpfs, model)
    model_stats[model]['% Solved'] = calculate_perc_solved(df_rpfs, model)  # given all instances
    for instance_type in instance_type_list:  # group by instance type and size
        df_itype = df_rpfs.reset_index()
        df_itype = df_itype[(df_itype['instance_type'] == instance_type)]
        instance_size_list = ['10x2', '10x3', '10x4', '10x5', '15x5']  # df_itype['instance_size'].unique().tolist()
        for instance_size in instance_size_list:
            model_stats[model]['% Solved '+ instance_size] = calculate_perc_solved(df_rpfs, model, instance_type, instance_size)
    model_stats[model]['Avg. % gap'] = calculate_avg_perc_gap(df_rpfs, model)
    model_stats[model]['Median time'] = calculate_median_time(df_rpfs, model, 'time')
    model_stats[model]['Median time MP'] = calculate_median_time(df_rpfs, model, 'mp_total_time')
    model_stats[model]['Median time SP'] = calculate_median_time(df_rpfs, model, 'sp_total_time')
    model_stats[model]['Median iterations'] = calculate_median_iterations(df_rpfs, model)

In [ ]:
model_stats

In [ ]:
model_stats_df = pd.DataFrame.from_dict(model_stats)
model_stats_df

### Export to Tableau, after melt

In [ ]:
model_stats_df.transpose().reset_index()

In [ ]:
value_vars = model_stats_df.transpose().columns
df_melt_model_stats_df = pd.melt(model_stats_df.transpose().reset_index(), id_vars=['index'], value_vars=value_vars)
df_melt_model_stats_df['Model'] = df_melt_model_stats_df['index']
df_melt_model_stats_df.to_excel(os.path.join(outputfolder_table, 'twct_model_stats.xlsx'))

In [ ]:
# Save output table as HTML
pd.set_option('colheader_justify', 'center')   # FOR TABLE <th>

html_string = '''
<html>
  <head><title>HTML Pandas Dataframe with CSS</title></head>
  <link rel="stylesheet" type="text/css" href="df_style.css"/>
  <body>
    {table}
  </body>
</html>.
'''

# OUTPUT AN HTML FILE
with open(os.path.join(outputfolder_table, 'model_stats.html'), 'w') as f:
    f.write(html_string.format(table=model_stats_df.to_html(classes='mystyle')))

## Table 3. Performance per instance group and model

Model-wise Robust PFSP C&CG performance comparison, per instance group.

* % Best Performance is the percentage of instances solved to optimality where the model achieved shorter execution time, when compared to the other models; 

* % Solved contains the percentage of instances solved within the time limit; 

* Avg. % Gap is the average percentage gap of solutions from instances not solved to optimality; 

* Avg. time and Std. dev. of time are the mean and standard deviation in solution time (s), respectively;

* Avg. iterations and Std. dev. of iterations are the mean and standard deviation of the number of iterations performed.

In [ ]:
df_itype['instance_size'].unique().tolist()

### First, let's create a table only with non-hybrid solution methods

In [ ]:
per_instance_stats = dict()
for instance_type in instance_type_list:  # group by instance type and size
    df_base = df_rpfs.reset_index()
    exclude_model_list = ['Wilson-Hybrid', 'Liao-You-Hybrid']
    df_base = df_base[~(df_base['model'].isin(exclude_model_list))]
    model_list_reduced = [_ for _ in model_list if _ not in exclude_model_list]
    df_itype = df_base
    df_itype = df_itype[(df_itype['instance_type'] == instance_type)]
    instance_size_list = ['10x2', '10x3', '10x4', '10x5', '15x5']  # df_itype['instance_size'].unique().tolist()
    for instance_size in instance_size_list:
        df_instance = df_itype[df_itype['instance_size'] == instance_size]
        for model in model_list_reduced:
            per_instance_stats[(instance_type,instance_size,model)] = dict()
            per_instance_stats[(instance_type,instance_size,model)]['% Best Performance'] = calculate_perc_best_performance(df_instance, model)
            per_instance_stats[(instance_type,instance_size,model)]['% Solved'] = calculate_perc_solved(df_base, model, instance_type, instance_size)
            per_instance_stats[(instance_type,instance_size,model)]['Avg. % gap'] = calculate_avg_perc_gap(df_instance, model)
            per_instance_stats[(instance_type,instance_size,model)]['Avg. time'] = calculate_avg_time(df_instance, model, 'time')
            per_instance_stats[(instance_type,instance_size,model)]['Std. dev. of time'] = calculate_std_time(df_instance, model, 'time')
            per_instance_stats[(instance_type,instance_size,model)]['Avg. MP time'] = calculate_avg_time(df_instance, model, 'mp_total_time')
            per_instance_stats[(instance_type,instance_size,model)]['Avg. SP time'] = calculate_avg_time(df_instance, model, 'sp_total_time')
            per_instance_stats[(instance_type,instance_size,model)]['Avg. iterations'] = calculate_avg_iterations(df_instance, model)
            per_instance_stats[(instance_type,instance_size,model)]['Std. dev. of iterations'] = calculate_std_iterations(df_instance, model)

In [ ]:
# https://stackoverflow.com/questions/57606801/pandas-style-options-to-latex

In [ ]:
pd.set_option('display.max_columns', None)
allowed_keys = [(x, y, z) for (x, y, z) in per_instance_stats.keys() if (x == 'ying' and y in ['10x2', '10x3'])]
per_instance_stats1 = { your_key: per_instance_stats[your_key] for your_key in allowed_keys }
df_table3a = pd.DataFrame.from_dict(per_instance_stats1)
df_table3a.columns = df_table3a.columns.droplevel()
df_table3a

In [ ]:
pd.set_option('display.max_columns', None)
allowed_keys = [(x, y, z) for (x, y, z) in per_instance_stats.keys() if (x == 'ying' and y in ['10x4', '10x5'])]
per_instance_stats2 = { your_key: per_instance_stats[your_key] for your_key in allowed_keys }
df_table3b = pd.DataFrame.from_dict(per_instance_stats2)
df_table3b.columns = df_table3b.columns.droplevel()
df_table3b

#### Export table to Tableau, after melt

In [ ]:
df_table3 = pd.DataFrame.from_dict(per_instance_stats)
df_table3.columns = df_table3.columns.droplevel()
value_vars = df_table3.transpose().columns
df_melt_table3 = pd.melt(df_table3.transpose().reset_index(), id_vars=['level_0', 'level_1'], value_vars=value_vars)
df_melt_table3['Instance size'] = df_melt_table3['level_0']
df_melt_table3['Model'] = df_melt_table3['level_1']
df_melt_table3.to_excel(os.path.join(outputfolder_table, 'twct_table3.xlsx'))

In [ ]:
# Save output table as HTML
pd.set_option('colheader_justify', 'center')   # FOR TABLE <th>

html_string = '''
<html>
  <head><title>HTML Pandas Dataframe with CSS</title></head>
  <link rel="stylesheet" type="text/css" href="df_style.css"/>
  <body>
    {table}
  </body>
</html>.
'''

# OUTPUT AN HTML FILE
with open(os.path.join(outputfolder_table, 'instance_stats_1.html'), 'w') as f:
    f.write(html_string.format(table=df_table3a.to_html(classes='mystyle')))
with open(os.path.join(outputfolder_table, 'instance_stats_2.html'), 'w') as f:
    f.write(html_string.format(table=df_table3b.to_html(classes='mystyle')))

### Now, we'll create a table with hybrid and non-hybrid solution methods

In [ ]:
per_instance_stats = dict()
for instance_type in instance_type_list:  # group by instance type and size
    df_itype = df_rpfs.reset_index()
    df_itype = df_itype[(df_itype['instance_type'] == instance_type)]
    instance_size_list = ['10x2', '10x3', '10x4', '10x5', '15x5']  # df_itype['instance_size'].unique().tolist()
    for instance_size in instance_size_list:
        df_instance = df_itype[df_itype['instance_size'] == instance_size]
        for model in model_list:
            per_instance_stats[(instance_type,instance_size,model)] = dict()
            per_instance_stats[(instance_type,instance_size,model)]['% Best Performance'] = calculate_perc_best_performance(df_instance, model)
            per_instance_stats[(instance_type,instance_size,model)]['% Solved'] = calculate_perc_solved(df_rpfs, model, instance_type, instance_size)
            per_instance_stats[(instance_type,instance_size,model)]['Avg. % gap'] = calculate_avg_perc_gap(df_instance, model)
            per_instance_stats[(instance_type,instance_size,model)]['Avg. time'] = calculate_avg_time(df_instance, model, 'time')
            per_instance_stats[(instance_type,instance_size,model)]['Std. dev. of time'] = calculate_std_time(df_instance, model, 'time')
            per_instance_stats[(instance_type,instance_size,model)]['Avg. MP time'] = calculate_avg_time(df_instance, model, 'mp_total_time')
            per_instance_stats[(instance_type,instance_size,model)]['Avg. SP time'] = calculate_avg_time(df_instance, model, 'sp_total_time')
            per_instance_stats[(instance_type,instance_size,model)]['Avg. iterations'] = calculate_avg_iterations(df_instance, model)
            per_instance_stats[(instance_type,instance_size,model)]['Std. dev. of iterations'] = calculate_std_iterations(df_instance, model)

In [ ]:
# https://stackoverflow.com/questions/57606801/pandas-style-options-to-latex

In [ ]:
pd.set_option('display.max_columns', None)
allowed_keys = [(x, y, z) for (x, y, z) in per_instance_stats.keys() if (x == 'ying' and y in ['10x2', '10x3', '10x4'])]
per_instance_stats1 = { your_key: per_instance_stats[your_key] for your_key in allowed_keys }
df_table3a = pd.DataFrame.from_dict(per_instance_stats1)
df_table3a

In [ ]:
pd.set_option('display.max_columns', None)
allowed_keys = [(x, y, z) for (x, y, z) in per_instance_stats.keys() if (x == 'ying' and y in ['10x5', '15x5'])]
per_instance_stats2 = { your_key: per_instance_stats[your_key] for your_key in allowed_keys }
df_table3b = pd.DataFrame.from_dict(per_instance_stats2)
df_table3b

In [ ]:
# Save output table as HTML
pd.set_option('colheader_justify', 'center')   # FOR TABLE <th>

html_string = '''
<html>
  <head><title>HTML Pandas Dataframe with CSS</title></head>
  <link rel="stylesheet" type="text/css" href="df_style.css"/>
  <body>
    {table}
  </body>
</html>.
'''

# OUTPUT AN HTML FILE
with open(os.path.join(outputfolder_table, 'instance_stats_1.html'), 'w') as f:
    f.write(html_string.format(table=df_table3a.to_html(classes='mystyle')))
with open(os.path.join(outputfolder_table, 'instance_stats_2.html'), 'w') as f:
    f.write(html_string.format(table=df_table3b.to_html(classes='mystyle')))